# Author Modeling Pipeline

This notebook implements a 5-stage pipeline for modeling an author's characteristic writing patterns through few-shot example curation.

## Methodology

Rather than extracting explicit style rules, this approach models the author's **decision-making patterns** and **sensibility**, then curates exemplary passages for tacit transmission via few-shot learning.

## Pipeline Stages

1. **Stage 1: Analytical Mining** - Analyze sample texts through three lenses:
   - Implied Author: Sensibility and stance
   - Decision Patterns: Compositional choices
   - Functional Texture: How surface features serve purpose

2. **Stage 2: Cross-Text Synthesis** - Synthesize each dimension across multiple samples to identify stable patterns

3. **Stage 3: Field Guide Construction** - Integrate all three syntheses into unified recognition criteria and density rubric

4. **Stage 4: Passage Evaluation** - Evaluate passages from corpus using field guide (1-5 density rating)

5. **Stage 5: Example Set Construction** - Curate 3-4 high-density passages that balance density, diversity, and complementarity

## Note

This pipeline is author-agnostic. The Russell corpus is used as a test case, but the same approach applies to any author.

## 1. Setup & Dependencies

In [1]:
!pip install -r requirements.txt

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from pathlib import Path
from pprint import pprint
from typing import List, Dict, Any

from belletrist import (
    LLM,
    LLMConfig,
    PromptMaker,
    DataSampler,
    ResultStore,
    # Stage 1 configs
    ImpliedAuthorConfig,
    DecisionPatternConfig,
    FunctionalTextureConfig,
    # Stage 2 configs
    ImpliedAuthorSynthesisConfig,
    DecisionPatternSynthesisConfig,
    TexturalSynthesisConfig,
    # Stage 3 config
    AuthorModelDefinitionConfig,
    FieldGuideConstructionConfig,
    # Stage 4 config
    PassageEvaluationConfig,
    # Stage 5 config
    ExampleSetConstructionConfig,
    # Utilities
    extract_paragraph_windows,
    extract_logical_sections,
    get_full_sample_as_passage,
    parse_passage_evaluation,
    parse_example_set_selection,
    validate_passage_evaluation,
    validate_example_set_selection,
)

print("Dependencies imported successfully")

Dependencies imported successfully


### Initialize Base Objects

In [3]:
# Initialize LLM
llm_config = LLMConfig(
    model="mistral/mistral-large-2411",  # Change as needed
    api_key=os.environ.get('MISTRAL_API_KEY'),
    temperature=0.7
)
llm = LLM(llm_config)

# Initialize prompt maker
prompt_maker = PromptMaker()

# Initialize data sampler (Russell corpus)
data_dir = Path("data/russell")
sampler = DataSampler(data_dir)

# Initialize result store
store = ResultStore("russell_author_modeling.db")

In [4]:
store.reset('all')

### Configuration: Select Samples for Analysis

For Stages 1-3, we'll analyze 3-5 samples to extract stable cross-text patterns.

In [6]:
# Select samples for Stage 1-3 analysis
# These should be diverse, substantial passages (500-800 words recommended)
ANALYSIS_SAMPLES = [
    "sample_001",
    "sample_002",
    "sample_003",
    "sample_004",
    "sample_005",
    # Add more as needed for richer cross-text synthesis
]

# For this demo, we'll generate samples if they don't exist
# In production, you might use specific pre-selected samples
NUM_SAMPLES = 5
SAMPLE_PARAGRAPH_LENGTH = 10

print(f"Will analyze {NUM_SAMPLES} samples through Stages 1-3")

Will analyze 5 samples through Stages 1-3


## 2. Stage 1: Analytical Mining

Run three specialized analyses on each sample:
- **Implied Author**: What sensibility emerges from the prose?
- **Decision Patterns**: What compositional choices are made at key junctures?
- **Functional Texture**: How do surface features serve purpose?

### Generate or Load Samples

In [13]:
# Generate samples if they don't exist
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    
    # Skip if already saved
    if store.get_sample(sample_id):
        print(f"✓ {sample_id} already gathered")
        continue
        
    # Generate new sample
    segment = sampler.sample_segment(p_length=SAMPLE_PARAGRAPH_LENGTH)
    store.save_segment(sample_id, segment)
    print(f"Generated {sample_id}")

# Display first sample for reference
sample = store.get_sample("sample_001")
print(f"\nSample 001 preview:\n{sample['text']}...")

✓ sample_001 already gathered
✓ sample_002 already gathered
✓ sample_003 already gathered
✓ sample_004 already gathered
✓ sample_005 already gathered

Sample 001 preview:
PART I

THE PRESENT CONDITION OF RUSSIA

I

WHAT IS HOPED FROM BOLSHEVISM

To understand Bolshevism it is not sufficient to know facts; it is
necessary also to enter with sympathy or imagination into a new
spirit. The chief thing that the Bolsheviks have done is to create a
hope, or at any rate to make strong and widespread a hope which was
formerly confined to a few. This aspect of the movement is as easy to
grasp at a distance as it is in Russia--perhaps even easier, because
in Russia present circumstances tend to obscure the view of the
distant future. But the actual situation in Russia can only be
understood superficially if we forget the hope which is the motive
power of the whole. One might as well describe the Thebaid without
mentioning that the hermits expected eternal bliss as the reward of
their sacrifices h

### Run Stage 1A: Implied Author Analysis

In [14]:
# Analyze each sample for implied author
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analyst_name = ImpliedAuthorConfig.analyst_name()
    
    # Check if analysis already exists (resume support)
    if store.get_analysis(sample_id, analyst_name):
        print(f"{sample_id}: {analyst_name} analysis already complete")
        continue
    
    # Get sample text
    sample = store.get_sample(sample_id)
    
    # Generate prompt
    config = ImpliedAuthorConfig(text=sample['text'])
    prompt = prompt_maker.render(config)
    
    # Run LLM
    print(f"{sample_id}: Running {analyst_name} analysis...")
    response = llm.complete(prompt)
    
    # Save analysis
    store.save_analysis(
        sample_id=sample_id,
        analyst=analyst_name,
        output=response.content,
        model=response.model
    )
    print(f"{sample_id}: {analyst_name} analysis complete")

print("\nStage 1A complete: All samples analyzed for implied author")

sample_001: Running implied_author analysis...
sample_001: implied_author analysis complete
sample_002: Running implied_author analysis...
sample_002: implied_author analysis complete
sample_003: Running implied_author analysis...
sample_003: implied_author analysis complete
sample_004: Running implied_author analysis...
sample_004: implied_author analysis complete
sample_005: Running implied_author analysis...
sample_005: implied_author analysis complete

Stage 1A complete: All samples analyzed for implied author


### Run Stage 1B: Decision Pattern Analysis

In [15]:
# Analyze each sample for decision patterns
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analyst_name = DecisionPatternConfig.analyst_name()
    
    if store.get_analysis(sample_id, analyst_name):
        print(f"{sample_id}: {analyst_name} analysis already complete")
        continue
    
    sample = store.get_sample(sample_id)
    config = DecisionPatternConfig(text=sample['text'])
    prompt = prompt_maker.render(config)
    
    print(f"{sample_id}: Running {analyst_name} analysis...")
    response = llm.complete(prompt)
    
    store.save_analysis(
        sample_id=sample_id,
        analyst=analyst_name,
        output=response.content,
        model=response.model
    )
    print(f"{sample_id}: {analyst_name} analysis complete")

print("\nStage 1B complete: All samples analyzed for decision patterns")

sample_001: Running decision_pattern analysis...
sample_001: decision_pattern analysis complete
sample_002: Running decision_pattern analysis...
sample_002: decision_pattern analysis complete
sample_003: Running decision_pattern analysis...
sample_003: decision_pattern analysis complete
sample_004: Running decision_pattern analysis...
sample_004: decision_pattern analysis complete
sample_005: Running decision_pattern analysis...
sample_005: decision_pattern analysis complete

Stage 1B complete: All samples analyzed for decision patterns


### Run Stage 1C: Functional Texture Analysis

In [16]:
# Analyze each sample for functional texture
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analyst_name = FunctionalTextureConfig.analyst_name()
    
    if store.get_analysis(sample_id, analyst_name):
        print(f"{sample_id}: {analyst_name} analysis already complete")
        continue
    
    sample = store.get_sample(sample_id)
    config = FunctionalTextureConfig(text=sample['text'])
    prompt = prompt_maker.render(config)
    
    print(f"{sample_id}: Running {analyst_name} analysis...")
    response = llm.complete(prompt)
    
    store.save_analysis(
        sample_id=sample_id,
        analyst=analyst_name,
        output=response.content,
        model=response.model
    )
    print(f"{sample_id}: {analyst_name} analysis complete")

print("\nStage 1C complete: All samples analyzed for functional texture")

sample_001: Running functional_texture analysis...
sample_001: functional_texture analysis complete
sample_002: Running functional_texture analysis...
sample_002: functional_texture analysis complete
sample_003: Running functional_texture analysis...
sample_003: functional_texture analysis complete
sample_004: Running functional_texture analysis...
sample_004: functional_texture analysis complete
sample_005: Running functional_texture analysis...
sample_005: functional_texture analysis complete

Stage 1C complete: All samples analyzed for functional texture


### Inspect Stage 1 Results

In [20]:
# Display completion status
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    sample, analyses = store.get_sample_with_analyses(sample_id)
    print(f"{sample_id}: {len(analyses)} analyses complete")
    for analyst_name in analyses.keys():
        print(f"  - {analyst_name}")

# Optionally display one analysis
print("\n--- Sample Implied Author Analysis (first 500 chars) ---")
sample, analyses = store.get_sample_with_analyses("sample_005")
implied_author = analyses.get(ImpliedAuthorConfig.analyst_name(), "Not found")
print(f"Total chars: {len(implied_author)}")
print(implied_author)

sample_001: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author
sample_002: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author
sample_003: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author
sample_004: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author
sample_005: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author

--- Sample Implied Author Analysis (first 500 chars) ---
Total chars: 6370
### PART 1: DIMENSIONAL ANALYSIS

#### 1. RELATIONSHIP TO MATERIAL

**Observation**: The author writes with a blend of curiosity and mastery, often seeking to unravel complexities and share insights. They approach the material with a keen intellectual interest, breaking down complex ideas into digestible parts.

**Evidence**:
1. "The bent stick in water belongs here. People say it looks bent but is straight: this only means that it is straight t

## 3. Stage 2: Cross-Text Synthesis

Synthesize each analytical dimension across samples to identify stable patterns.

### Stage 2A: Implied Author Synthesis

In [22]:
# Collect all implied author analyses
implied_author_analyses = {}
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analysis = store.get_analysis(sample_id, ImpliedAuthorConfig.analyst_name())
    if analysis:
        implied_author_analyses[sample_id] = analysis

# Check if synthesis already exists
synthesis_type = ImpliedAuthorSynthesisConfig.synthesis_type()
existing = store.list_syntheses(synthesis_type)

if existing:
    print(f"Implied author synthesis already exists: {existing[0]}")
    implied_author_synthesis_id = existing[0]
else:
    # Generate synthesis prompt
    config = ImpliedAuthorSynthesisConfig(
        implied_author_analyses=implied_author_analyses
    )
    prompt = prompt_maker.render(config)
    
    # Run synthesis
    print("Running implied author synthesis...")
    response = llm.complete(prompt)
    
    # Save synthesis
    sample_contributions = [
        (sample_id, ImpliedAuthorConfig.analyst_name())
        for sample_id in implied_author_analyses.keys()
    ]
    
    implied_author_synthesis_id = store.save_synthesis(
        synthesis_type=synthesis_type,
        output=response.content,
        model=response.model,
        sample_contributions=sample_contributions,
        config=config
    )
    print(f"Synthesis saved: {implied_author_synthesis_id}")

# Display preview
synth = store.get_synthesis(implied_author_synthesis_id)
print(f"\nPreview (first 400 chars):\n{synth['output'][:400]}...")

Implied author synthesis already exists: implied_author_synthesis_001

Preview (first 400 chars):
## PART 1: THE STABLE CORE

### Fundamental Stance Toward Ideas and Inquiry

**How does this author consistently engage with their subject matter?**
The author consistently engages with their subject matter through a blend of curiosity, mastery, and critical distance. They display a deep intellectual interest, systematically exploring complex phenomena and seeking to dissolve them through logical ...


### Stage 2B: Decision Pattern Synthesis

In [24]:
# Collect all decision pattern analyses
decision_pattern_analyses = {}
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analysis = store.get_analysis(sample_id, DecisionPatternConfig.analyst_name())
    if analysis:
        decision_pattern_analyses[sample_id] = analysis

# Check if synthesis already exists
synthesis_type = DecisionPatternSynthesisConfig.synthesis_type()
existing = store.list_syntheses(synthesis_type)

if existing:
    print(f"Decision pattern synthesis already exists: {existing[0]}")
    decision_pattern_synthesis_id = existing[0]
else:
    config = DecisionPatternSynthesisConfig(
        decision_pattern_analyses=decision_pattern_analyses
    )
    prompt = prompt_maker.render(config)
    
    print("Running decision pattern synthesis...")
    response = llm.complete(prompt)
    
    sample_contributions = [
        (sample_id, DecisionPatternConfig.analyst_name())
        for sample_id in decision_pattern_analyses.keys()
    ]
    
    decision_pattern_synthesis_id = store.save_synthesis(
        synthesis_type=synthesis_type,
        output=response.content,
        model=response.model,
        sample_contributions=sample_contributions,
        config=config
    )
    print(f"Synthesis saved: {decision_pattern_synthesis_id}")

synth = store.get_synthesis(decision_pattern_synthesis_id)
print(f"\nPreview (first 400 chars):\n{synth['output'][:400]}...")

Running decision pattern synthesis...
Synthesis saved: decision_pattern_synthesis_001

Preview (first 400 chars):
## PART 1: COMPOSITIONAL PROBLEM TAXONOMY

Across all texts, the following types of compositional problems recur for this author:

1. **Opening a text or major section**
   - **Description**: The author often faces the challenge of introducing complex topics in a way that sets the tone and frames the discussion effectively. This involves selecting the right opening strategy, whether it's a philoso...


### Stage 2C: Textural Synthesis

In [25]:
# Collect all functional texture analyses
textural_analyses = {}
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analysis = store.get_analysis(sample_id, FunctionalTextureConfig.analyst_name())
    if analysis:
        textural_analyses[sample_id] = analysis

# Check if synthesis already exists
synthesis_type = TexturalSynthesisConfig.synthesis_type()
existing = store.list_syntheses(synthesis_type)

if existing:
    print(f"Textural synthesis already exists: {existing[0]}")
    textural_synthesis_id = existing[0]
else:
    config = TexturalSynthesisConfig(
        textural_analyses=textural_analyses
    )
    prompt = prompt_maker.render(config)
    
    print("Running textural synthesis...")
    response = llm.complete(prompt)
    
    sample_contributions = [
        (sample_id, FunctionalTextureConfig.analyst_name())
        for sample_id in textural_analyses.keys()
    ]
    
    textural_synthesis_id = store.save_synthesis(
        synthesis_type=synthesis_type,
        output=response.content,
        model=response.model,
        sample_contributions=sample_contributions,
        config=config
    )
    print(f"Synthesis saved: {textural_synthesis_id}")

synth = store.get_synthesis(textural_synthesis_id)
print(f"\nPreview (first 400 chars):\n{synth['output'][:400]}...")

Running textural synthesis...
Synthesis saved: textural_synthesis_001

Preview (first 400 chars):
## SYNTHESIS OF TEXTURAL ANALYSES

## PART 1: SENTENCE ARCHITECTURE

### Structural Character

This author's characteristic sentence is typically complex and varied, often employing long, compound-complex structures that are either front-loaded or back-loaded. The main clauses often precede a series of subordinate clauses that add layers of detail and qualification, creating a sense of progression...


In [ ]:
implied_synth = store.get_synthesis(implied_author_synthesis_id)
decision_synth = store.get_synthesis(decision_pattern_synthesis_id)
textural_synth = store.get_synthesis(textural_synthesis_id)

# Check if author model definition already exists
synthesis_type = AuthorModelDefinitionConfig.synthesis_type()
existing = store.list_syntheses(synthesis_type)

if existing:
    print(f"Author model definition already exists: {existing[0]}")
    author_model_id = existing[0]
else:
    # Generate author model definition
    config = AuthorModelDefinitionConfig(
        implied_author_synthesis=implied_synth['output'],
        decision_pattern_synthesis=decision_synth['output'],
        textural_synthesis=textural_synth['output']
    )
    prompt = prompt_maker.render(config)

    print("Constructing author model definition...")
    response = llm.complete(prompt)

    # Save with parent linkage (no direct sample contributions)
    author_model_id = store.save_synthesis(
        synthesis_type=synthesis_type,
        output=response.content,
        model=response.model,
        sample_contributions=[],  # Inherits from parents
        config=config,
        parent_synthesis_id=implied_author_synthesis_id  # Link to one parent
    )
    print(f"Author model definition saved: {author_model_id}")

# Display preview
author_model = store.get_synthesis(author_model_id)
print(f"\nTotal chars of Author Model: {len(author_model['output'])}")
print(f"\nAuthor Model Preview (first 800 chars):\n{author_model['output'][:800]}...")

# Export to filesystem for use in generation
output_dir = Path("outputs/author_modeling")
output_dir.mkdir(parents=True, exist_ok=True)

author_model_path = output_dir / f"{author_model_id}.txt"
store.export_synthesis(
    synthesis_id=author_model_id,
    output_path=author_model_path,
    metadata_format='yaml'
)
print(f"\nAuthor model exported to: {author_model_path}")

## 4. Stage 3: Field Guide Construction

Integrate all three Stage 2 syntheses into a unified recognition field guide with:
- Unified sensibility description
- Recognition criteria (questions for evaluation)
- Density rubric (1-5 scale)
- Master passage index

In [ ]:
# Get the three syntheses
implied_synth = store.get_synthesis(implied_author_synthesis_id)
decision_synth = store.get_synthesis(decision_pattern_synthesis_id)
textural_synth = store.get_synthesis(textural_synthesis_id)

# Check if field guide already exists
synthesis_type = FieldGuideConstructionConfig.synthesis_type()
existing = store.list_syntheses(synthesis_type)

if existing:
    print(f"Field guide already exists: {existing[0]}")
    field_guide_id = existing[0]
else:
    # Generate field guide
    config = FieldGuideConstructionConfig(
        implied_author_synthesis=implied_synth['output'],
        decision_pattern_synthesis=decision_synth['output'],
        textural_synthesis=textural_synth['output']
    )
    prompt = prompt_maker.render(config)
    
    print("Constructing field guide...")
    response = llm.complete(prompt)
    
    # Save with parent linkage (no direct sample contributions)
    field_guide_id = store.save_synthesis(
        synthesis_type=synthesis_type,
        output=response.content,
        model=response.model,
        sample_contributions=[],  # Inherits from parents
        config=config,
        parent_synthesis_id=implied_author_synthesis_id  # Link to one parent
    )
    print(f"Field guide saved: {field_guide_id}")

# Display preview
field_guide = store.get_synthesis(field_guide_id)
print(f"\nTotal chars of Field Guide:{len(field_guide['output'])}")
print(f"\nField Guide Preview (first 500 chars):\n{field_guide['output'][:500]}...")

## 5. Corpus Mining Preparation

Extract passages from the corpus for evaluation in Stage 4.

We can use several strategies:
- **Paragraph windows**: Overlapping sliding windows
- **Logical sections**: Section-based extraction
- **Full samples**: Evaluate entire samples
- **Specific indices**: Passages flagged in Stage 1-3

In [ ]:
# Strategy 1: Extract paragraph windows (blind sliding window)
passages_windows = extract_paragraph_windows(
    store,
    sample_ids=None,
    window_size=4,
    overlap=2
)
print(f"Strategy 1 (Sliding Windows): {len(passages_windows)} passages")

# Strategy 2: Extract logical sections
passages_sections = extract_logical_sections(
    store,
    sample_ids=None,
    min_length=200,
    max_length=600
)
print(f"Strategy 2 (Logical Sections): {len(passages_sections)} passages")

# Strategy 3: Extract nominated passages from Stage 1-2 analyses
# These are passages that analysts flagged as signature moments or high-density examples
passages_nominated = []

for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    
    # Extract from each analysis type
    for analyst_config, analysis_type in [
        (ImpliedAuthorConfig, "implied_author"),
        (DecisionPatternConfig, "decision_pattern"),
        (FunctionalTextureConfig, "functional_texture")
    ]:
        analysis = store.get_analysis(sample_id, analyst_config.analyst_name())
        if analysis:
            try:
                nominated = extract_nominated_passages_from_analysis(
                    store=store,
                    llm=llm,
                    sample_id=sample_id,
                    analysis_text=analysis,
                    analysis_type=analysis_type,
                    padding_before=2,
                    padding_after=2,
                    min_confidence=0.6
                )
                passages_nominated.extend(nominated)
                print(f"  {sample_id}/{analysis_type}: {len(nominated)} nominated passages")
            except Exception as e:
                print(f"  Warning: Failed to extract from {sample_id}/{analysis_type}: {e}")

print(f"\nStrategy 3 (Nominated from Analyses): {len(passages_nominated)} passages")

# Choose evaluation strategy
EVALUATION_STRATEGY = "nominated_only"  # Change to "combined" or "all"

if EVALUATION_STRATEGY == "nominated_only":
    passages_to_evaluate = passages_nominated
elif EVALUATION_STRATEGY == "combined":
    passages_to_evaluate = passages_nominated + passages_windows[:5]
else:  # "all"
    passages_to_evaluate = passages_nominated + passages_windows[:5] + passages_sections[:5]

print(f"\nUsing strategy: {EVALUATION_STRATEGY}")
print(f"Will evaluate {len(passages_to_evaluate)} passages in Stage 4")


In [ ]:
# VALIDATION: Test quote extraction on sample_001
print("=== QUOTE EXTRACTION VALIDATION ===\n")

sample_id = "sample_001"
analysis = store.get_analysis(sample_id, "implied_author")

if analysis:
    from belletrist.models.author_modeling_models import QuoteExtractionConfig
    
    config = QuoteExtractionConfig(
        analysis_text=analysis,
        analysis_type="implied_author"
    )
    prompt = prompt_maker.render(config)
    
    response = llm.complete_json(prompt)
    quote_data = json.loads(response.content)
    
    print(f"Extracted {len(quote_data['quotes'])} quotes from implied_author analysis")
    
    # Test fuzzy matching on first quote
    if quote_data['quotes']:
        first_quote = quote_data['quotes'][0]['quote_text']
        print(f"\nTesting fuzzy match for first quote:")
        print(f"  Quote (first 80 chars): {first_quote[:80]}...")
        
        result = find_passage_by_quote(store, sample_id, first_quote)
        
        if result.found:
            print(f"  ✓ Match found!")
            print(f"  Confidence: {result.match_confidence:.2f}")
            print(f"  Core range: {result.core_range}")
            print(f"  Full range (with padding): {result.full_range}")
        else:
            print("  ✗ Match failed")
else:
    print(f"No implied_author analysis found for {sample_id}")


## 6. Stage 4: Passage Evaluation

Evaluate passages using the field guide to assign density ratings (1-5) and assess few-shot suitability.

In [ ]:
# Get field guide for evaluation
field_guide = store.get_synthesis(field_guide_id)
field_guide_text = field_guide['output']

# Evaluate each passage
evaluation_results = []

for idx, passage in enumerate(passages_to_evaluate):
    print(f"Evaluating passage {idx+1}/{len(passages_to_evaluate)}...")
    
    # Create passage identifier
    passage_id = f"{passage['source_sample_id']}_para_{passage['paragraph_range']}"
    
    # Check if evaluation already exists
    # (In production, you'd check ResultStore for existing evaluations)
    
    # Generate evaluation prompt
    config = PassageEvaluationConfig(
        field_guide=field_guide_text,
        source=passage_id,
        passage=passage['text']
    )
    prompt = prompt_maker.render(config)
    
    # Run evaluation
    response = llm.complete(prompt)
    
    # Parse evaluation response
    try:
        parsed = parse_passage_evaluation(response.content)
        
        if validate_passage_evaluation(parsed):
            # Save to ResultStore
            eval_id = store.save_passage_evaluation(
                sample_id=passage['source_sample_id'],
                density_rating=parsed['density_rating'],
                task_coverage=parsed.get('task_coverage', ''),
                teaching_value=parsed.get('teaching_value', ''),
                recommendation=parsed['recommendation'],
                model=response.model,
                field_guide_id=field_guide_id,
                paragraph_range=passage['paragraph_range']
            )
            
            evaluation_results.append({
                'eval_id': eval_id,
                'passage_id': passage_id,
                'rating': parsed['density_rating'],
                'recommended': parsed['recommendation']
            })
            
            print(f"  → Rating: {parsed['density_rating']}, Recommended: {parsed['recommendation']}")
        else:
            print(f"  → Validation failed for passage {idx+1}")
    
    except Exception as e:
        print(f"  → Error parsing evaluation: {e}")

print(f"\nStage 4 complete: Evaluated {len(evaluation_results)} passages")

### Inspect Evaluation Results

In [ ]:
# Display rating distribution
rating_counts = {}
for result in evaluation_results:
    rating = result['rating']
    rating_counts[rating] = rating_counts.get(rating, 0) + 1

print("Rating Distribution:")
for rating in sorted(rating_counts.keys()):
    print(f"  Rating {rating}: {rating_counts[rating]} passages")

# Show high-density passages (4-5 rated)
high_density = [r for r in evaluation_results if r['rating'] >= 4]
print(f"\nHigh-density passages (4-5): {len(high_density)}")
for result in high_density:
    print(f"  {result['eval_id']}: Rating {result['rating']}")

## 7. Stage 5: Example Set Construction

Curate 3-4 high-density passages that balance:
- **Density**: High ratings (4-5)
- **Diversity**: Different compositional tasks/topics
- **Complementarity**: Each adds something unique
- **Coherence**: Together forming unified demonstration

In [ ]:
# Get all passage evaluations for curation
# (In production, query ResultStore for evaluations with ratings 4-5)
high_density_evals = [r for r in evaluation_results if r['rating'] >= 4]

if len(high_density_evals) < 3:
    print(f"Warning: Only {len(high_density_evals)} high-density passages found")
    print("Consider evaluating more passages or lowering density threshold")
else:
    # Collect full evaluation texts for each high-density passage
    passage_evaluations = []
    for result in high_density_evals:
        eval_data = store.get_passage_evaluation(result['eval_id'])
        if eval_data:
            # Format as evaluation text for the prompt
            eval_text = f"""Passage ID: {result['eval_id']}
Density Rating: {eval_data['density_rating']}
Recommendation: {'YES' if eval_data['recommendation'] else 'NO'}

Task Coverage:
{eval_data.get('task_coverage', 'N/A')}

Teaching Value:
{eval_data.get('teaching_value', 'N/A')}
"""
            passage_evaluations.append(eval_text)
    
    # Generate example set curation prompt
    config = ExampleSetConstructionConfig(
        field_guide=field_guide_text,
        passage_evaluations=passage_evaluations
    )
    prompt = prompt_maker.render(config)
    
    # Run curation
    print(f"Curating example set from {len(passage_evaluations)} high-density passages...")
    response = llm.complete(prompt)
    
    # Parse selected passages
    try:
        selected = parse_example_set_selection(response.content)
        
        if validate_example_set_selection(selected):
            # Save example set to ResultStore
            example_set_id = store.save_example_set(
                field_guide_id=field_guide_id,
                selection_rationale=response.content,  # Full curation response
                model=response.model
            )
            
            # Add member passages
            for passage in selected:
                store.add_example_set_member(
                    example_set_id=example_set_id,
                    evaluation_id=passage['passage_id'],
                    unique_contribution=passage.get('unique_contribution', '')
                )
            
            print(f"\nExample set saved: {example_set_id}")
            print(f"Selected passages:")
            for p in selected:
                print(f"  - {p['passage_id']} (rating: {p.get('density_rating', 'N/A')})")
        else:
            print("Validation failed: Example set does not meet criteria")
    
    except Exception as e:
        print(f"Error parsing example set selection: {e}")

## 8. Export & Inspection

Export field guide and example sets to filesystem for use in generation tasks.

In [ ]:
# Create output directory
output_dir = Path("outputs/author_modeling")
output_dir.mkdir(parents=True, exist_ok=True)

# Export field guide
field_guide_path = output_dir / f"{field_guide_id}.txt"
store.export_synthesis(
    synthesis_id=field_guide_id,
    output_path=field_guide_path,
    metadata_format='yaml'
)
print(f"Field guide exported to: {field_guide_path}")

# Export example set (if exists)
example_sets = store.list_example_sets()
if example_sets:
    for example_set_id in example_sets:
        # Get example set with members
        example_set = store.get_example_set_with_members(example_set_id)
        
        # Write to file
        example_set_path = output_dir / f"{example_set_id}.txt"
        with open(example_set_path, 'w') as f:
            f.write(f"# Example Set: {example_set_id}\n\n")
            f.write(f"Field Guide: {example_set['field_guide_id']}\n")
            f.write(f"Created: {example_set['created_at']}\n")
            f.write(f"Model: {example_set['model']}\n\n")
            
            f.write("## Selection Rationale\n\n")
            f.write(example_set['selection_rationale'])
            f.write("\n\n## Selected Passages\n\n")
            
            for member in example_set['members']:
                f.write(f"### {member['evaluation_id']}\n\n")
                f.write(f"**Unique Contribution:** {member['unique_contribution']}\n\n")
                
                # Get actual passage text
                eval_data = store.get_passage_evaluation(member['evaluation_id'])
                if eval_data:
                    sample = store.get_sample(eval_data['sample_id'])
                    # Extract paragraph range from full sample
                    # (simplified - in production, implement proper extraction)
                    f.write(f"**Source:** {eval_data['sample_id']}\n")
                    f.write(f"**Paragraph Range:** {eval_data.get('paragraph_range', 'N/A')}\n")
                    f.write(f"**Density Rating:** {eval_data['density_rating']}\n\n")
                f.write("---\n\n")
        
        print(f"Example set exported to: {example_set_path}")
else:
    print("No example sets to export")

print("\nExport complete")

## 9. Provenance Verification

Verify full audit trail from samples through to example sets.

In [ ]:
# Get field guide provenance
print("Field Guide Provenance:")
provenance = store.get_synthesis_provenance(field_guide_id)
pprint(provenance, depth=3)

# Get example set provenance
if example_sets:
    print(f"\nExample Set {example_sets[0]} Provenance:")
    example_set = store.get_example_set_with_members(example_sets[0])
    print(f"  Field Guide: {example_set['field_guide_id']}")
    print(f"  Members: {len(example_set['members'])}")
    for member in example_set['members']:
        eval_data = store.get_passage_evaluation(member['evaluation_id'])
        print(f"    - {member['evaluation_id']} (from {eval_data['sample_id']})")

print("\nProvenance verification complete")

## Summary

This notebook implemented a complete author modeling pipeline:

1. ✅ **Stage 1**: Analyzed samples through three lenses (implied author, decision patterns, functional texture)
2. ✅ **Stage 2**: Synthesized each dimension across samples
3. ✅ **Stage 3**: Constructed unified field guide with recognition criteria and rubric
4. ✅ **Stage 4**: Evaluated passages using field guide (density ratings 1-5)
5. ✅ **Stage 5**: Curated 3-4 high-density passages as few-shot example set

**Next Steps:**
- Use exported field guide for understanding author's patterns
- Use example set for few-shot learning in generation tasks
- Compare few-shot results against baseline methods (Stage 6 - future work)

All results stored in `russell_author_modeling.db` with full provenance tracking.